In [10]:
import json
import os
import django
import sys
sys.path.append('..')
os.environ.setdefault('DJANGO_SETTINGS_MODULE', 'SMARTROLL.settings')
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"
os.environ.setdefault('DJANGO_SETTINGS_MODULE', 'SMARTROLL.settings')
django.setup()
from Manage.models import College,Term,Branch,Stream,Semester,Division,Batch,Subject,TimeTable,GPSCoordinates,Classroom,Schedule,Lecture,Link,PermanentSubject,SubjectChoices
from Session.models import Attendance,Session
from StakeHolders.models import NotificationSubscriptions,SuperAdmin,Admin,Teacher,Student,Profile
from django.contrib.auth import get_user_model
import datetime
import pandas as pd
import string

In [11]:
file_path='/home/manav1011/Documents/SMARTROLL_SSIP_2023/SMARTROLL/notebooks/sheets/electives (2).xlsx'

In [12]:
sheet_names = pd.ExcelFile(file_path).sheet_names
ML = pd.read_excel(file_path, sheet_name='ML')
ML_COUNT = ML['enrollment'].count()
print(f"ML Student Count - {ML_COUNT}")
BDA = pd.read_excel(file_path, sheet_name='BDA')
BDA_COUNT = BDA['enrollment'].count()
print(f"BDA Student Count - {BDA_COUNT}")
IS = pd.read_excel(file_path, sheet_name='IS')
IS_COUNT = IS['enrollment'].count()
print(f"IS Student Count - {IS_COUNT}")
MCWC = pd.read_excel(file_path, sheet_name='MCWC')
MCWC_COUNT = MCWC['enrollment'].count()
print(f"MCWC Student Count - {MCWC_COUNT}")
CC = pd.read_excel(file_path, sheet_name='CC')
CC_COUNT = CC['enrollment'].count()
print(f"CC Student Count - {CC_COUNT}")
MAD = pd.read_excel(file_path, sheet_name='MAD')
MAD_COUNT = MAD['enrollment'].count()
print(f"MAD Student Count - {MAD_COUNT}")
DF = pd.read_excel(file_path, sheet_name='DF')
DF_COUNT = DF['enrollment'].count()
print(f"DF Student Count - {DF_COUNT}")


ML Student Count - 118
BDA Student Count - 60
IS Student Count - 178
MCWC Student Count - 24
CC Student Count - 154
MAD Student Count - 93
DF Student Count - 85


In [13]:
def update_student_subjects(df, subject_name,student_subjects):
    for _, row in df.iterrows():
        student_key = (row['enrollment'], row['name'])  # Create a tuple of (enrollment, name)
        if student_key in student_subjects:
            student_subjects[student_key].append(subject_name)
        else:
            student_subjects[student_key] = [subject_name]


In [14]:
student_subjects = {}
update_student_subjects(ML, 'ML',student_subjects)
update_student_subjects(BDA, 'BDA',student_subjects)
update_student_subjects(IS, 'IS',student_subjects)
update_student_subjects(MCWC, 'MCWC',student_subjects)
update_student_subjects(CC, 'CC',student_subjects)
update_student_subjects(MAD, 'MAD',student_subjects)
update_student_subjects(DF, 'DF',student_subjects)

In [15]:
min_div_size = 60
max_div_size = 90
def calculate_divisions(student_count):    
    min_divisions = (student_count + max_div_size - 1) // max_div_size
    max_divisions = student_count // min_div_size        
    for divisions in range(min_divisions, max_divisions + 1):
        div_size = student_count // divisions
        if min_div_size <= div_size <= max_div_size:
            return divisions        
    return min_divisions

In [16]:
total_hours = {
    "ML":5,
    "BDA":5,
    "IS":5,
    "MCWC":3,
    "CC":3,
    "MAD":4,
    "DF":4,
}
total_student_count = len(student_subjects.keys())
approx_division_count = calculate_divisions(total_student_count)
# the total teaching our for each subject will be in multiplication of the approx division count

# for subject in total_hours:
#     total_hours[subject] *= approx_division_count
# total_hours

In [17]:
subject_groups = {}
for student, subject_group in student_subjects.items():
    # Convert subject_group to a tuple if it isn't one already
    subject_group_tuple = tuple(subject_group)
    if subject_group_tuple not in subject_groups:
        subject_groups[subject_group_tuple] = [student]
    else:
        subject_groups[subject_group_tuple].append(student)

In [18]:
for subject_group in subject_groups:
    print(f"Group - {subject_group} | Student Count {len(subject_groups[subject_group])}")

Group - ('ML', 'IS', 'CC', 'DF') | Student Count 69
Group - ('ML', 'IS', 'CC', 'MAD') | Student Count 44
Group - ('ML', 'IS', 'MCWC', 'MAD') | Student Count 5
Group - ('BDA', 'IS', 'MCWC', 'MAD') | Student Count 19
Group - ('BDA', 'IS', 'CC', 'MAD') | Student Count 25
Group - ('BDA', 'IS', 'CC', 'DF') | Student Count 16


In [ ]:
BDA,MAD,MCWC
ML,IS,CC

In [86]:
divisions = {}
for i in range(approx_division_count):
    divisions[string.ascii_uppercase[i]] = []
divisions

{'A': [], 'B': []}

In [87]:
remaining_students_count = total_student_count
for division in divisions:        
    # check the count of students in current division
    students_count_in_div = len(divisions[division])
    capacity_remained_in_div = max_div_size-students_count_in_div
    if capacity_remained_in_div >= remaining_students_count:
        # insert all the remaining students in current div
        for subject_group,students in subject_groups.items():
            divisions[division] += students
    else:
        # iterate through the subject groups
        # check if any available subject group has the student count greater than the max_div_count
        subject_group_gt_max_div_count= list(filter(lambda subject_group: len(subject_groups[subject_group]) > max_div_size,subject_groups))
        if subject_group_gt_max_div_count:
            # move the first max_div_count students to this division
            divisions[division] = subject_groups[subject_group_gt_max_div_count[0]][:max_div_size]
            subject_groups[subject_group_gt_max_div_count[0]] = subject_groups[subject_group_gt_max_div_count[0]][max_div_size:]
            # also reduce the remaining student count            
            remaining_students_count -= max_div_size
            # now this div's full so move let's move to next div            
            continue
            

In [88]:
# Now to sort the students according to their enrollment in each divisions
for division in divisions:
    divisions[division]=sorted(divisions[division], key=lambda student: student[0])
divisions

{'A': [(200280107145, 'Prajapati Jay Kumar Dashrathbhai '),
  (210280107001, 'Ratadiya Mayur Devsibhai '),
  (210280107003, 'Taral Bambhania'),
  (210280107004, 'Darshil Nareshbhai Patel'),
  (210280107005, 'Akshar Chauhan'),
  (210280107006, 'Patel Nisarg Surendrakumar'),
  (210280107007, 'DOBARIYA HAPPY RAMNIKBHAI '),
  (210280107008, 'Meet Soni '),
  (210280107009, 'Nirali Darji'),
  (210280107010, 'Devmurari Maulik Vijaybhai'),
  (210280107011, 'Justin George'),
  (210280107012, 'Patel Dhruvit Vasantbhai '),
  (210280107013, 'Halpati Divya DipakKumar '),
  (210280107014, 'Jiyani Hasti Mukeshbhai'),
  (210280107015, 'Astha Hareshbhai Viradiya '),
  (210280107016, 'Kaman Devanshi KaushikKumar '),
  (210280107017, 'Naik Chinmay Dharmesh'),
  (210280107018, 'Patel Chirag Sunilbhai'),
  (210280107019, 'PRAJAPATI HARSH DHIRENDRAKUMAR '),
  (210280107020, 'KERALIYA KARN NAROTTAMBHAI'),
  (210280107022, 'Henish Kantibhai Naliyadhara '),
  (210280107024, 'Jha Kunjbihari Vinodanand '),
  (21

In [1]:
# Sample data for student groups and their counts
student_groups = { 
    ('ML', 'IS', 'CC', 'DF'): 138,
    ('ML', 'IS', 'CC', 'MAD'): 6,
    ('ML', 'IS', 'MCWC', 'MAD'): 2,
    ('ML', 'IS', 'MCWC', 'DF'): 3,
    ('BDA', 'IS', 'MCWC', 'MAD'): 9,
    ('BDA', 'IS', 'CC', 'DF'): 9,
    ('BDA', 'IS', 'CC', 'MAD'): 1,
    ('BDA', 'IS', 'MCWC', 'DF'): 10,
}

# GTU minimum teaching hours for each subject
total_hours = {
    "ML": 4,
    "BDA": 4,
    "IS": 4,
    "MCWC": 3,
    "CC": 3,
    "MAD": 3,
    "DF": 3,
}

# Max students per division
MAX_STUDENTS_PER_DIVISION = 80

# Function to split groups if the count exceeds the max allowed per division
def split_groups(groups, max_students):
    split_groups = {}
    for group, count in groups.items():
        part = 1
        while count > max_students:
            split_groups[(group, f"part{part}")] = max_students
            count -= max_students
            part += 1
        if count > 0:
            split_groups[(group, f"part{part}")] = count
    return split_groups

# Split the student groups if any group exceeds MAX_STUDENTS_PER_DIVISION
student_groups = split_groups(student_groups, MAX_STUDENTS_PER_DIVISION)

# Calculate the number of divisions needed based on total students
total_students = sum(student_groups.values())
num_divisions = (total_students // MAX_STUDENTS_PER_DIVISION) + (1 if total_students % MAX_STUDENTS_PER_DIVISION != 0 else 0)

# Initialize division allocations
divisions = {i: [] for i in range(num_divisions)}  # A list of student groups for each division
division_sizes = {i: 0 for i in range(num_divisions)}  # Number of students in each division

# Initialize teaching hours for each subject in each division
teaching_hours_per_division = {i: {subject: 0 for subject in total_hours} for i in range(num_divisions)}

# Backtracking function to try placing groups in divisions
def backtrack(group_idx, student_groups, division_sizes, divisions, teaching_hours_per_division):
    # Base case: All groups are assigned
    if group_idx == len(student_groups):
        # Check if all subjects meet the required hours in each division
        for subject, min_hours in total_hours.items():
            total_hours_assigned = sum(teaching_hours_per_division[d][subject] for d in range(num_divisions))
            if total_hours_assigned < min_hours:
                return False
        return True

    group, student_count = list(student_groups.items())[group_idx]
    subjects = group[0]  # group[0] is the original tuple of subjects
    
    # Try placing this group in any division (while respecting the max student limit)
    for div in range(num_divisions):
        if division_sizes[div] + student_count <= MAX_STUDENTS_PER_DIVISION:
            # Tentatively add the group to the division
            division_sizes[div] += student_count
            for subject in subjects:
                teaching_hours_per_division[div][subject] += student_count
            
            # Recurse to place the next group
            if backtrack(group_idx + 1, student_groups, division_sizes, divisions, teaching_hours_per_division):
                # If valid, assign this group to the division
                divisions[div].append((group, student_count))
                return True
            
            # If placing the group here didn't work, backtrack
            division_sizes[div] -= student_count
            for subject in subjects:
                teaching_hours_per_division[div][subject] -= student_count

    return False

# Call the backtracking function to find a solution
if backtrack(0, student_groups, division_sizes, divisions, teaching_hours_per_division):
    print("Optimal Assignment of Student Groups to Divisions:")
    for div in range(num_divisions):
        print(f"Division {div + 1}:")
        for group, student_count in divisions[div]:
            print(f"  Group {group[0]} ({group[1]}) with {student_count} students")
                    
    print("\nFinal Teaching Hours per Subject:")
    for subject in total_hours:
        total_hours_assigned = sum(teaching_hours_per_division[d][subject] for d in range(num_divisions))
        print(f"{subject}: {total_hours_assigned} hours (Minimum required: {total_hours[subject]} hours)")
else:
    print("No valid assignment found.")

Optimal Assignment of Student Groups to Divisions:
Division 1:
  Group ('ML', 'IS', 'CC', 'DF') (part1) with 80 students
Division 2:
  Group ('BDA', 'IS', 'CC', 'MAD') (part1) with 1 students
  Group ('BDA', 'IS', 'MCWC', 'MAD') (part1) with 9 students
  Group ('ML', 'IS', 'MCWC', 'DF') (part1) with 3 students
  Group ('ML', 'IS', 'MCWC', 'MAD') (part1) with 2 students
  Group ('ML', 'IS', 'CC', 'MAD') (part1) with 6 students
  Group ('ML', 'IS', 'CC', 'DF') (part2) with 58 students
Division 3:
  Group ('BDA', 'IS', 'MCWC', 'DF') (part1) with 10 students
  Group ('BDA', 'IS', 'CC', 'DF') (part1) with 9 students

Final Teaching Hours per Subject:
ML: 149 hours (Minimum required: 4 hours)
BDA: 29 hours (Minimum required: 4 hours)
IS: 178 hours (Minimum required: 4 hours)
MCWC: 24 hours (Minimum required: 3 hours)
CC: 154 hours (Minimum required: 3 hours)
MAD: 18 hours (Minimum required: 3 hours)
DF: 160 hours (Minimum required: 3 hours)
